# Instantiate the map centered on SECOORA

In [1]:
import folium

lon, lat = -86.276, 30.935 
zoom_start = 5

mapa = folium.Map(location=[lat, lon], tiles='Cartodb Positron',
                  zoom_start=zoom_start)

# Create the GIS database from the spreadsheet

In [2]:
import os

fname = 'hfradar/hfradar.geojson'
if os.path.isfile(fname):
    os.unlink(fname)

%pushd hfradar/

%run convert.py
%popd

/home/filipe/IOOS/SECOORA/static_assets/hfradar
/home/filipe/IOOS/SECOORA/static_assets
popd -> ~/IOOS/SECOORA/static_assets


In [3]:
fname = 'stations/'
if os.path.isfile(fname):
    os.unlink(fname)

%pushd stations/
%run convert.py
%popd

/home/filipe/IOOS/SECOORA/static_assets/stations
/home/filipe/IOOS/SECOORA/static_assets
popd -> ~/IOOS/SECOORA/static_assets


# Stations

In [4]:
import json

with open('stations/stations.geojson') as f:
    stations = json.load(f)


icon_size = (14, 14)

for feature in stations['features']:
    lon, lat = feature['geometry']['coordinates']
    icon_url = feature['properties']['icon']
    popup = feature['properties']['popupcontent']
    
    icon = folium.features.CustomIcon(icon_url,
                                      icon_size=(14, 14))
    
    marker = folium.map.Marker([lat, lon], icon=icon,
                               popup=folium.map.Popup(popup))
    mapa.add_children(marker)

# HF-Radar

In [5]:
with open('hfradar/hfradar.geojson') as f:
    hfradar = json.load(f)

    
style_function = lambda feature: dict(fillColor='#DEFFDE',
                                      color='#AECCAE',
                                      weight=1,
                                      opacity=0.6,
                                      dashArray='0')

for feature in hfradar['features']:
    if feature['geometry']['type'] == 'Point':
        lon, lat = feature['geometry']['coordinates']
        icon_url = feature['properties']['icon']
        popup = feature['properties']['popupcontent']
        
        icon = folium.features.CustomIcon(icon_url,
                                          icon_size=(14, 14))
        marker = folium.map.Marker([lat, lon], icon=icon,
                                   popup=folium.map.Popup(popup))
        mapa.add_children(marker)
    elif feature['geometry']['type'] == 'Polygon':
        gjson = folium.features.GeoJson(feature,
                                        style_function=style_function)
        mapa.add_children(gjson)

In [6]:
mapa